In [2]:
from os import listdir
from os.path import isfile, join
import cv2
import random
import numpy as  np
import math

def generateExpandedImage(image):
    # This function generates a white background image that is 104% larger (four times of sin(15))than the original image
    # In this program max rotation degree is +-15 degrees. sin(15 degree)<0.258
    (h, w) = image.shape[:2]
    (h_new, w_new) = (math.ceil(h * 2.1), math.ceil(w * 2.1))
    # print ("h = {}, w = {}, h_new = {}, w_new = {}".format(h,w, h_new, w_new))
    bg_img = np.zeros(shape=[h_new, w_new], dtype=np.uint8)
    bg_img = cv2.bitwise_not(bg_img)
    top_left_y = int((h_new - h)/2)
    top_left_x = int((w_new - w)/2)
    # print ("top_left_y = {}, top_left_x = {}".format(top_left_y,top_left_x))
    bg_img[top_left_y:h+top_left_y, top_left_x:w+top_left_x]=image
    return (bg_img, h, w)
    
def rotateImage(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def cutbackImage(image, h, w):
    (h_r,w_r) = image.shape[:2]
    h_c, w_c = math.ceil(h * 1.26), math.ceil(w * 1.26)
    # 1.26 comes from sin(15) = 02586
    # print ("h = {}, w = {}, h_c = {}, w_c = {}".format(h,w, h_c, w_c))
    # specify the area of top lef to lower right
    # math.ceil rounds up number into integer
    image_cut = image[math.ceil((h_r - h_c)/2):h_c+math.ceil((h_r - h_c)/2), math.ceil((w_r - w_c)/2):w_c+math.ceil((w_r - w_c)/2)]
    return image_cut, h_c, w_c

def resizeImage(image, h, w, t):
    w_resize = math.ceil(w * t / h)  # maintain the aspect ratio
    dim = (w_resize, t)
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return result

def top_left (max_v, v):  # this function calculate top_left coordiates and works for both x and y
    rand = np.random.randn()  # generate random number with average 0 starndard diviation 1
    while abs(rand) > 3:   # eliminate rare cases that exceed 3 sigma
        rand = np.random.randn()
    v_range = 2 * (math.floor(max_v/2)-math.ceil(v/2))
    if v_range < 0:  # manage rounding error
        v_range = 0
    # since random number generate between -3 and +3, the following equation divide the possible range by 6
    top_left = math.floor(max_v/2) - math.floor(v_range/6*rand)- math.floor(v/2)  
    return top_left


def placeImage (timage, oimage, max_h, max_w): # place the character to max size randomely
    bg_img = np.zeros(shape=[max_h, max_w], dtype=np.uint8)
    bg_img = cv2.bitwise_not(bg_img)
    (th,tw), (oh,ow) = timage.shape[:2], oimage.shape[:2]
    # print("th = {}, tw = {}, oh ={}, ow ={}".format(th, tw, oh, ow))
    if tw + ow > 132:
        timage = resizeFinal(timage, math.floor(th*132/(tw+ow)), math.floor(tw*132/(tw+ow)))
        oimage = resizeFinal(oimage, math.floor(oh*132/(tw+ow)), math.floor(ow*132/(tw+ow)))
        (th,tw), (oh,ow) = timage.shape[:2], oimage.shape[:2]
    if tw + ow < 130:
        gap = int(np.random.randn()*(130 - (tw+ow))/4+10)
        if gap < 0:
            gap = 0
        if gap > 130 - (tw+ow):
            gap = 130 - (tw+ow)
    else:
        gap = 0
    tl_h = top_left(max_h, th)
    tl_w = top_left(max_w, tw+ow+gap)
    # print ("h = {}, w = {}, tl_h ={}, tl_w={}".format(h,w,tl_h,tl_w))
    bg_img[tl_h:th+tl_h, tl_w:tw+tl_w] = timage
    bg_img[tl_h:oh+tl_h, tl_w+tw+gap:ow+tl_w+tw+gap] = oimage
    return bg_img
    
def resizeFinal(image, h, w):
    dim = (w, h)
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    result = cv2.bitwise_not(result)
    return result

def listFileNames(rpath):
    onlyfolders = [f for f in listdir(rpath) if f[-4:] != ".ini"]
    for folder in onlyfolders:
        npath = rpath +"/"+ folder
        # print(npath)
        filenames = [f for f in listdir(npath) if isfile(join(npath, f)) and f[-4:] == ".png"]
        files[folder] = filenames
    return files

def rotatedImage(rpath, num, h_target, files):
    tpath = rpath + "/" + str(num) + "/" + random.choice(files[str(num)])
    # print(tpath)
    image = cv2.imread(tpath, 0)
    angle = random.randint(-15, 15) # range of rotation -15 to 15 degrees
    bg_img, h, w = generateExpandedImage(image) # add extra white blank around the original image
    result = rotateImage(bg_img, angle) # rotate image
    image, h, w = cutbackImage(result, h, w)  # cut unnecessary area and extract the rotated character
    result = resizeImage(image, h, w, h_target) # randomly resize the image
    return result
    
rpath = "F:/Google Drive/User_Backup/base_font"
wpath = "F:/Google Drive/User_Backup/training"
h_max, w_max, h_out, w_out = 75, 132, 25, 44
files = {}
files = listFileNames(rpath)
for j in range (10, 100):
    tens, ones = math.floor(j/10), j%10
    print(j)
    for k in range (5000):
        h_target = random.randint(40, 75)
        timage = rotatedImage(rpath, tens, h_target, files)
        oimage = rotatedImage(rpath, ones, h_target, files)
        result = placeImage(timage, oimage, h_max, w_max)
        result = resizeFinal(result, h_out, w_out) # shrink the final image for AI model training
        opath = wpath + "/" + str(j) + "/" + str(k) + ".png"        
        cv2.imwrite(opath,result)        
        # cv2.imshow('image', result)
        # cv2.waitKey()
        # cv2.destroyAllWindows()
        # print(opath)    
    
    



10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
